## import all the necessary libraries

In [8]:
import pandas as pd 
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import warnings
#Ignoring unnecessory warnings
warnings.filterwarnings("ignore") 

#### Read dataset and show shape.

In [9]:
df=pd.read_csv('G:\\BOOK\\MY Field\\My Person Project\\NLP Project 1\\Reviews.csv')

In [13]:
#read first 10 columns
df.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


### Now we can shaow information for dataset

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


**now thea data give us some info like it have a 568454 entries and 10 columns and it haven't any missingvalue and it consist from object and int64**

#### we can show the columns


In [15]:
df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

**we can now show if this dataset have any missing value or not.**

In [16]:
df.isna().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

### Data Preparation

In [29]:
### add some columns for upvote metrics
df['Helpful %'] = np.where(df['HelpfulnessDenominator'] > 0, df['HelpfulnessNumerator'] / df['HelpfulnessDenominator'], -1)

In [32]:
df.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Text,Helpful %
0,1,B001E4KFG0,A3SGXH7AUHU8GW,1,1,5,1303862400,I have bought several of the Vitality canned d...,1.0
1,2,B00813GRG4,A1D87F6ZCVE5NK,0,0,1,1346976000,Product arrived labeled as Jumbo Salted Peanut...,-1.0
2,3,B000LQOCH0,ABXLMWJIXXAIN,1,1,4,1219017600,This is a confection that has been around a fe...,1.0
3,4,B000UA0QIQ,A395BORC6FGVXV,3,3,2,1307923200,If you are looking for the secret ingredient i...,1.0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,0,0,5,1350777600,Great taffy at a great price. There was a wid...,-1.0


#### assigning different labels to helpful% according to its value

In [33]:
df['Helpful %'].unique()

array([ 1.        , -1.        ,  0.8       ,  0.        ,  0.5       ,
        0.66666667,  0.25      ,  0.89473684,  0.83333333,  0.75      ,
        0.33333333,  0.3       ,  0.11111111,  0.42857143,  0.875     ,
        0.85714286,  0.2       ,  0.26315789,  0.6       ,  0.71428571,
        0.53846154,  0.57142857,  0.91489362,  0.86666667,  0.82352941,
        0.78571429,  0.74074074,  0.4       ,  0.375     ,  0.28571429,
        0.14285714,  0.77777778,  0.125     ,  0.9       ,  0.94117647,
        0.92307692,  0.7       ,  0.45454545,  0.88888889,  0.83870968,
        0.9047619 ,  0.92857143,  0.90909091,  0.91666667,  0.84615385,
        0.10526316,  0.98214286,  0.97826087,  0.7518797 ,  0.3125    ,
        0.1       ,  0.18518519,  0.88      ,  0.69230769,  0.625     ,
        0.54545455,  0.41666667,  0.45833333,  0.22222222,  0.81818182,
        0.8125    ,  0.16666667,  0.93103448,  0.88235294,  0.23529412,
        0.63636364,  0.81481481,  0.95652174,  0.64285714,  0.58

In [34]:
pd.cut(df['Helpful %'] , bins = [-1, 0, 0.2, 0.4, 0.6, 0.8, 1.0], labels = 
       ['Empty', '0-20%', '20-40%', '40-60%', '60-80%', '80-100%'])

0         80-100%
1             NaN
2         80-100%
3         80-100%
4             NaN
           ...   
568449        NaN
568450        NaN
568451    80-100%
568452    80-100%
568453        NaN
Name: Helpful %, Length: 568454, dtype: category
Categories (6, object): ['Empty' < '0-20%' < '20-40%' < '40-60%' < '60-80%' < '80-100%']

In [35]:
df['%upvote'] = pd.cut( df['Helpful %'] , bins = [-1, 0, 0.2, 0.4, 0.6, 0.8, 1.0], labels = ['Empty', '0-20%', '20-40%', '40-60%', '60-80%', '80-100%'])

In [36]:
df.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Text,Helpful %,%upvote
0,1,B001E4KFG0,A3SGXH7AUHU8GW,1,1,5,1303862400,I have bought several of the Vitality canned d...,1.0,80-100%
1,2,B00813GRG4,A1D87F6ZCVE5NK,0,0,1,1346976000,Product arrived labeled as Jumbo Salted Peanut...,-1.0,NaN
2,3,B000LQOCH0,ABXLMWJIXXAIN,1,1,4,1219017600,This is a confection that has been around a fe...,1.0,80-100%
3,4,B000UA0QIQ,A395BORC6FGVXV,3,3,2,1307923200,If you are looking for the secret ingredient i...,1.0,80-100%
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,0,0,5,1350777600,Great taffy at a great price. There was a wid...,-1.0,NaN


In [37]:
df.groupby(['Score', '%upvote']).agg('count')

Id  ProductId  UserId  HelpfulnessNumerator  \
Score %upvote                                                    
1     Empty      8060       8060    8060                  8060   
      0-20%      2338       2338    2338                  2338   
      20-40%     4649       4649    4649                  4649   
      40-60%     6586       6586    6586                  6586   
      60-80%     5838       5838    5838                  5838   
      80-100%   12531      12531   12531                 12531   
2     Empty      4234       4234    4234                  4234   
      0-20%       762        762     762                   762   
      20-40%     1618       1618    1618                  1618   
      40-60%     3051       3051    3051                  3051   
      60-80%     2486       2486    2486                  2486   
      80-100%    7014       7014    7014                  7014   
3     Empty      5062       5062    5062                  5062   
      0-20%       474        474     474                   474   
      20-40%     1506       1506    1506                  1506   
      40-60%     3384       3384    3384                  3384   
      60-80%     2754       2754    2754                  2754   
      80-100%   11037      11037   11037                 11037   
4     Empty      4780       4780    4780                  4780   
      0-20%       116        116     116                   116   
      20-40%      909        909     909                   909   
      40-60%     3185       3185    3185                  3185   
      60-80%     2941       2941    2941                  2941   
      80-100%   26707      26707   26707                 26707   
5     Empty     11638      11638   11638                 11638   
      0-20%       432        432     432                   432   
      20-40%     2275       2275    2275                  2275   
      40-60%    10312      10312   10312                 10312   
      60-80%    11060      11060   11060                 11060   
      80-100%  140661     140661  140661                140661   

               HelpfulnessDenominator    Time    Text  Helpful %  
Score %upvote                                                     
1     Empty                      8060    8060    8060       8060  
      0-20%                      2338    2338    2338       2338  
      20-40%                     4649    4649    4649       4649  
      40-60%                     6586    6586    6586       6586  
      60-80%                     5838    5838    5838       5838  
      80-100%                   12531   12531   12531      12531  
2     Empty                      4234    4234    4234       4234  
      0-20%                       762     762     762        762  
      20-40%                     1618    1618    1618       1618  
      40-60%                     3051    3051    3051       3051  
      60-80%                     2486    2486    2486       2486  
      80-100%                    7014    7014    7014       7014  
3     Empty                      5062    5062    5062       5062  
      0-20%                       474     474     474        474  
      20-40%                     1506    1506    1506       1506  
      40-60%                     3384    3384    3384       3384  
      60-80%                     2754    2754    2754       2754  
      80-100%                   11037   11037   11037      11037  
4     Empty                      4780    4780    4780       4780  
      0-20%                       116     116     116        116  
      20-40%                      909     909     909        909  
      40-60%                     3185    3185    3185       3185  
      60-80%                     2941    2941    2941       2941  
      80-100%                   26707   26707   26707      26707  
5     Empty                     11638   11638   11638      11638  
      0-20%                       432     432     432        432  
      20-40%                  